# Dark Sky API (Weather)

Answer the following questions using the [Dark Sky API](https://darksky.net/dev/). I've added three cells for each question but you're free to use more or less! Hold `Shift` and hit `Enter` to run a cell, and use the `+` on the top left to add a new cell to a notebook.

If you spend some time reading the documentation your life will probably be a little bit easier!

## 0) Import any libraries you might need

- *Tip: We're going to be downloading things from the internet, so we probably need `requests`.*

In [1]:
import requests

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
secretapikey = os.getenv("darksky_apikey")
base_url = "https://api.darksky.net/forecast/"

## 1) Make a request from the Dark Sky API for where you were born (or lived, or want to visit!).

- *Tip: The URL we used in class was for a place near San Francisco. What was the format of the endpoint that made this happen?*
- *Tip: Save the URL as a separate variable, and be sure to not have `[` and `]` inside.*
- *Tip: How is north vs. south and east vs. west latitude/longitude represented? Is it the normal North/South/East/West?*
- *Tip: You know it's JSON, but Python doesn't! Make sure you aren't trying to deal with plain text.* 
- *Tip: Once you've imported the JSON into a variable, check the timezone's name to make sure it seems like it got the right part of the world!*

In [3]:
latlon_phila = ("39.96"+","+"-75.18")
weather_phila = requests.get(f'{base_url}{secretapikey}/{latlon_phila}').json()
degree_sign = u'\N{DEGREE SIGN}'

In [4]:
# print(weather_phila.keys())

In [5]:
#weather_phila['daily']

In [6]:
now_phila = weather_phila['currently']
# now_phila

## 2) What's the current wind speed? How much warmer does it feel than it actually is?

- *Tip: You can do this by browsing through the dictionaries, but it might be easier to read the documentation*
- *Tip: For the second half: it **is** one temperature, and it **feels** a different temperature. Calculate the difference.*

In [7]:
print(f"Windspeed is {now_phila['windSpeed']:.1f} kpm (so {now_phila['windSpeed'] * 0.539957:.1f} knots, i guess?).")

Windspeed is 4.3 kpm (so 2.3 knots, i guess?).


In [10]:
now_temp = int(now_phila['temperature'])
now_tempapp = int(now_phila['apparentTemperature'])
now_tempdif = now_temp - now_tempapp
if now_tempdif > 0:
    feels = (f"{abs(now_tempdif)}{degree_sign} warmer.")
elif now_tempdif < 0:
    feels = (f"{abs(now_tempdif)}{degree_sign} cooler.")
else:
    feels = "just like it."
print(f"The temperature is currently {now_temp}{degree_sign} but it feels {feels}.")

The temperature is currently 72° but it feels 2° cooler..


## 3) Today's forecast is the FIRST daily forecast. For the place you decided on above, how much of the moon is currently visible?

- *Tip: This does **not** use the `currently` key.*
- *Tip: It might be helpful to save the forecast into its own variable*

In [11]:
daily_phila = weather_phila['daily']['data']
# daily_phila[0]

In [12]:
print(f"In its current phase, {int(daily_phila[0]['moonPhase'] * 100)}% of the moon should be visible.")

In its current phase, 30% of the moon should be visible.


## 4) What's the difference between the high and low temperatures for today?

In [13]:
tempdif = daily_phila[0]['temperatureHigh'] - daily_phila[0]['temperatureLow']

In [14]:
print(f"There's about a {abs(tempdif):.0f}{degree_sign} difference in today's high & low temps.")

There's about a 21° difference in today's high & low temps.


## 5) Go through the daily forecasts, printing out the next week's worth of predictions.

I'd like to know the **high temperature** for each day, and whether it's **hot, warm, or cold** (based on what temperatures you think are hot, warm or cold).

- *Tip: Is it returning F or C? You can change them by adding `?units=us` or `?units=si` onto the end of the URL.*
- *Tip: You'll need to use an `if` statement to say whether it is hot, warm or cold.*

In [15]:
from datetime import datetime
import calendar

In [16]:
for day in daily_phila:
    day['dt'] = datetime.fromtimestamp(day['time'])
    day['dt_clean'] = (day['dt']).strftime('%c').replace(' 00:00:00',',')
    day['dt_clean_long'] = (day['dt']).strftime('%A, %B %d')
#     print(day['dt'])
#     print(type(day['dt']))
#     print(day['dt_clean'])
#     print(type(day['dt_clean']))

In [17]:
print("This week's forecast is:")
for day in daily_phila:
    if day['temperatureMax'] >= 95:
        temp_desc = 'hot'
    elif float(day['temperatureMax']) >= 81:
        temp_desc = 'warm'
    elif float(day['temperatureMax']) >= 70:
        temp_desc = 'pleasant' 
    elif float(day['temperatureMax']) >= 45:
        temp_desc = 'cool'
    elif float(day['temperatureMax']) < 45:
        temp_desc = 'cold' 
    print(f"\t{day['dt_clean']}: {temp_desc}, high of {int(day['temperatureMax'])}{degree_sign}")

This week's forecast is:
	Tue Jun 11, 2019: pleasant, high of 79°
	Wed Jun 12, 2019: pleasant, high of 75°
	Thu Jun 13, 2019: pleasant, high of 73°
	Fri Jun 14, 2019: pleasant, high of 73°
	Sat Jun 15, 2019: warm, high of 83°
	Sun Jun 16, 2019: warm, high of 89°
	Mon Jun 17, 2019: warm, high of 90°
	Tue Jun 18, 2019: warm, high of 87°


## 6) What will be the hottest day in the next week? What is the high temperature on that day?

It's okay if your result looks like `1453363200`. If you want to be extra-fancy, though, format it as a **real date** - the weird number is called a **UNIX timestamp**, which might help in googling.

- *Tip: It's okay to skip this if it seems too difficult!*

In [18]:
week_highs_phila = [float(day['temperatureMax']) for day in daily_phila]
week_high_phila = max(week_highs_phila)

In [19]:
print(f"The warmest day of the week will be {day['dt_clean_long']} with a high of {int(week_high_phila)}{degree_sign}.")

The warmest day of the week will be Tuesday, June 18 with a high of 90°.


## 7) What's the weather looking like for the next 24+ hours in Miami, Florida?

I'd like to know the temperature for every hour, and if it's going to have cloud cover of more than 50% say "{temperature} and cloudy" instead of just the temperature. 


- *Tip: Be sure to remember how latitude and longitude are done with positive and negative numbers*
- *Tip: How do you represent "50%" for cloud cover?*

In [20]:
latlon_miami = ("25.76"+","+"-80.19")
weather_miami = requests.get(f'{base_url}{secretapikey}/{latlon_miami}').json()
hourly_miami = weather_miami['hourly']['data']
# hourly_miami[0]

In [21]:
for hour in hourly_miami:
    hour['dt'] = datetime.fromtimestamp(hour['time'])
    hour['dt_clean_long'] = (hour['dt']).strftime('%A, %B %d, %I%p')

In [22]:
print(f"Here is the weather forecast in Miami for the next 24 hours:")
for hour in hourly_miami   [:24]:
    ccover = hour['cloudCover'] * 100
    if ccover > 60:
        clarity = 'cloudy'
    if ccover > 40:
        clarity = 'mostly cloudy'
    if ccover > 10:
        clarity = 'mostly clear'
    else:
        clarity = 'clear'
    print(f"\t{hour['dt_clean_long']}: \t{int(hour['temperature'])}{degree_sign} and {clarity}")

Here is the weather forecast in Miami for the next 24 hours:
	Tuesday, June 11, 02AM: 	79° and clear
	Tuesday, June 11, 03AM: 	79° and clear
	Tuesday, June 11, 04AM: 	79° and mostly clear
	Tuesday, June 11, 05AM: 	79° and mostly clear
	Tuesday, June 11, 06AM: 	79° and mostly clear
	Tuesday, June 11, 07AM: 	80° and mostly clear
	Tuesday, June 11, 08AM: 	81° and mostly clear
	Tuesday, June 11, 09AM: 	84° and mostly clear
	Tuesday, June 11, 10AM: 	86° and mostly clear
	Tuesday, June 11, 11AM: 	88° and mostly clear
	Tuesday, June 11, 12PM: 	89° and mostly clear
	Tuesday, June 11, 01PM: 	90° and mostly clear
	Tuesday, June 11, 02PM: 	91° and mostly clear
	Tuesday, June 11, 03PM: 	90° and mostly clear
	Tuesday, June 11, 04PM: 	90° and mostly clear
	Tuesday, June 11, 05PM: 	89° and mostly clear
	Tuesday, June 11, 06PM: 	87° and mostly clear
	Tuesday, June 11, 07PM: 	86° and mostly clear
	Tuesday, June 11, 08PM: 	84° and mostly clear
	Tuesday, June 11, 09PM: 	83° and mostly clear
	Tuesday, Jun

## 8) For the next 24-ish hours in Miami, what percent of the time is the temperature above 85 degrees?

If they give you 48 hours instead of 24, that's okay.

- *Tip: You might want to read up on [looping patterns](http://jonathansoma.com/lede/foundations-2017/classes/data%20structures/looping-patterns/)*

In [23]:
hours_above = len([hour for hour in hourly_miami [:24] if float(hour['temperature']) > 85])
# print(hours_above)
print(f'The temperature in Miami will be above 85{degree_sign} for {(hours_above / 24) *100:.1f}% of the next 24 hours.')

The temperature in Miami will be above 85° for 41.7% of the next 24 hours.


## 9) What was the temperature in Central Park on Christmas Day, 2012? How about 2007? 2005? How far back does the API allow you to go?

- *Tip: You'll need to use UNIX time, which is the number of seconds since January 1, 1970. Google can help you convert a normal date!*
- *Tip: You'll want to use Forecast.io's "time machine" API, [documentation here](https://darksky.net/dev/docs/time-machine)*